In [1]:
import sys
import os

sys.path.insert(0, os.path.join(os.path.abspath(os.getcwd()), '..', '..'))
sys.path.insert(0, os.path.join(os.path.abspath(os.getcwd()), '..', '..', 'build', 'python'))

In [2]:
import pyspiel
from open_spiel.python.algorithms import cfr, exploitability
from open_spiel.python.algorithms import expected_game_score

In [3]:

game = pyspiel.load_game("universal_poker")

games_list = pyspiel.registered_games()

print("Registered games:")
print(games_list)


Registered games:
[<GameType 'backgammon'>, <GameType 'blotto'>, <GameType 'breakthrough'>, <GameType 'bridge_uncontested_bidding'>, <GameType 'catch'>, <GameType 'chess'>, <GameType 'coin_game'>, <GameType 'connect_four'>, <GameType 'coop_box_pushing'>, <GameType 'first_sealed_auction'>, <GameType 'go'>, <GameType 'goofspiel'>, <GameType 'havannah'>, <GameType 'hex'>, <GameType 'kuhn_poker'>, <GameType 'laser_tag'>, <GameType 'leduc_poker'>, <GameType 'liars_dice'>, <GameType 'markov_soccer'>, <GameType 'matching_pennies_3p'>, <GameType 'matrix_cd'>, <GameType 'matrix_coordination'>, <GameType 'matrix_mp'>, <GameType 'matrix_pd'>, <GameType 'matrix_rps'>, <GameType 'matrix_sh'>, <GameType 'matrix_shapleys_game'>, <GameType 'misere'>, <GameType 'negotiation'>, <GameType 'oshi_zumo'>, <GameType 'oware'>, <GameType 'pentago'>, <GameType 'phantom_ttt'>, <GameType 'pig'>, <GameType 'quoridor'>, <GameType 'tic_tac_toe'>, <GameType 'tiny_bridge_2p'>, <GameType 'tiny_bridge_4p'>, <GameType 't

In [ ]:
"""Test that Python and C++ bots can be called by a C++ algorithm."""

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

from absl.testing import absltest
import numpy as np

from open_spiel.python.bots import uniform_random
import pyspiel

game = pyspiel.load_game("leduc_poker")
bots = [
    pyspiel.make_uniform_random_bot(game, 0, 1234),
    uniform_random.UniformRandomBot(game, 1, np.random.RandomState(4321)),
]
results = np.array([
    pyspiel.evaluate_bots(game.new_initial_state(), bots, iteration)
    for iteration in range(10000)
])
leduc_average_results = np.mean(results, axis=0)
print(leduc_average_results)

game = pyspiel.load_game("universal_poker")
bots = [
    pyspiel.make_uniform_random_bot(game, 0, 1234),
    uniform_random.UniformRandomBot(game, 1, np.random.RandomState(4321)),
]
results = np.array([
    pyspiel.evaluate_bots(game.new_initial_state(), bots, iteration)
    for iteration in range(10000)
])
universal_poker_average_results = np.mean(results, axis=0)
print(universal_poker_average_results)
    
#np.testing.assert_allclose(universal_poker_average_results, leduc_average_results, atol=0.1)

In [ ]:
kuhnGameDesc = '''GAMEDEF
limit
numPlayers = 2
numRounds = 1
blind = 1 1
raiseSize = 1
firstPlayer = 1
maxRaises = 1
numSuits = 1
numRanks = 3
numHoleCards = 1
numBoardCards = 0
END GAMEDEF'''

In [ ]:

# flags.DEFINE_integer("iterations", 100, "Number of iterations")
# flags.DEFINE_string("game", "kuhn_poker", "Name of the game")
# flags.DEFINE_integer("players", 2, "Number of players")
# flags.DEFINE_integer("print_freq", 10, "How often to print the exploitability")

# Compare exloitability for two games

game_1 = "leduc_poker"
game_2 = "universal_poker"
players = 2
iterations = 1000
print_freq = 1

def compare_exploitability(game_1 = game_1, game_2 = game_2):
    game_1 = pyspiel.load_game(game_1,
                           {"players": pyspiel.GameParameter(players)})
    cfr_solver_1 = cfr.CFRSolver(game_1)
    game_2 = pyspiel.load_game(game_2)
    cfr_solver_2 = cfr.CFRSolver(game_2)
    
    for i in range(iterations):
        cfr_solver_1.evaluate_and_update_policy()
        cfr_solver_2.evaluate_and_update_policy()
        if i % print_freq == 0:
            conv_1 = exploitability.exploitability(game_1, cfr_solver_1.average_policy())
            conv_2 = exploitability.exploitability(game_2, cfr_solver_2.average_policy())


            print("Iteration {} exploitability of the {}: {} {}: {}".format(i, game_1, game_2, conv_1, conv_2))

    print("Final exploitability is {}: {} {}: {}".format(game_1, game_2, conv_1, conv_2))
    
compare_exploitability(game_1, game_2)

In [ ]:
import pyspiel
from open_spiel.python.visualizations import treeviz


In [ ]:
game = "universal_poker"
out = "/tmp/gametree.png"
prog = "dot"
group_infosets = False
group_terminal = False
verbose = False

def _zero_sum_node_decorator(state):
    """Custom node decorator that only shows the return of the first player."""
    attrs = treeviz.default_node_decorator(state)  # get default attributes
    if state.is_terminal():
        attrs["label"] = str(int(state.returns()[0]))
    return attrs


game = pyspiel.load_game(game, {'gameDesc':  pyspiel.GameParameter(ku)} )
game_type = game.get_type()

if game_type.dynamics != pyspiel.GameType.Dynamics.SEQUENTIAL:
    raise ValueError("Game must be sequential, not {}".format(
         game_type.dynamics))

if (game_type.utility == pyspiel.GameType.Utility.ZERO_SUM and
        game.num_players() == 2):
    gametree = treeviz.GameTree(
         game,
         node_decorator=_zero_sum_node_decorator,
         group_infosets=group_infosets,
         group_terminal=group_terminal)
else:
    gametree = treeviz.GameTree(game)  # use default decorators

if verbose:
     logging.info("Game tree:\n%s", gametree.to_string())

gametree.draw(out, prog=prog)

